In [1]:
from notebook_helper import *
import geopandas
from helpers import space, la

notebook_setup()

# Export normalised distances

Get normalised grid distance between all councils.

In [2]:

epsg = 7405 # british national grid

gdf = geopandas.read_file(Path("data", "geo", "gb", "district_borough_unitary_region.shp")).to_crs(epsg=epsg)
gdf = gdf[["CODE", "geometry"]]
ni = geopandas.read_file(Path("data", "geo", "ni", "OSNI_Open_Data_Largescale_Boundaries__Local_Government_Districts_2012.shp")).to_crs(epsg=epsg)
ni = ni[["LGDCode", "geometry"]].rename(columns={"LGDCode": "CODE"})
gdf = pd.concat([gdf, ni])
gdf["x"] = gdf["geometry"].centroid.x
gdf["y"] = gdf["geometry"].centroid.y

In [4]:

gdf = (gdf
       .la.create_code_column(from_type="gss", source_col="CODE")
       .drop(columns=["CODE"])
       )
distance = (gdf
            .space.self_distance("local-authority-code", ["x", "y"], normalize=True)
            .sort_values("distance"))
distance.to_csv(
    Path("data", "outputs", "geographic_distance.csv"), index=False)
distance.head()


,CODE_A,CODE_B,distance
131249,E09000020,E09000013,2.099896e+03
130876,E09000013,E09000020,2.099896e+03
131251,E09000020,E09000033,2.590818e+03
131624,E09000033,E09000020,2.590818e+03
132748,E09000019,E09000007,3.279809e+03
...,...,...,...
50995,S12000027,E06000026,1.127744e+06
49878,E06000052,S12000027,1.134221e+06
50997,S12000027,E06000052,1.134221e+06
50998,S12000027,E06000053,1.207212e+06
